In [1]:
import pandas as pd

## Data

In [18]:
import json

def read_json(file):
    with open(file) as f:
        return json.load(f)


In [19]:
df = pd.read_csv('eval_scenarios/reception_agent_adapt.csv',delimiter=";")
conditions = read_json('eval_scenarios/conditions.json')
goals = read_json('eval_scenarios/goals.json')

In [14]:
df.columns

Index(['num_condition', 'type', 'goals', 'GT_subgoals_disfavour',
       'GT_subgoals_favour'],
      dtype='object')

In [21]:
df['condition'] = df['num_condition'].apply(lambda x: conditions[str(x)])

In [25]:
# Función para seleccionar los subgoals correctos a partir de múltiples goals en una fila
def get_subgoals(row, conditions):
    condition = row['condition']
    goal_ids = eval(row['goals'])  # Suponemos que los ids de los goals están separados por comas
    subgoals_list = []
    
    # Iterar sobre todos los goal_ids y obtener los subgoals correspondientes
    for goal_id in goal_ids:
        goal_id = goal_id.strip()  # Eliminar espacios en blanco
        if 'human' in condition:
            subgoals_list.extend(goals[goal_id]['subgoals_human'])
        elif 'robot' in condition:
            subgoals_list.extend(goals[goal_id]['subgoals_robot'])
    
    return subgoals_list

# Crear la nueva columna con los subgoals concatenados en una lista
df['subgoals'] = df.apply(lambda row: get_subgoals(row, conditions), axis=1)

## Execute LLM

In [30]:
import openai
import os

from llm_env_to_goal import LLMEnvToGoalReasoner
from llm_agent_to_action import LLMAgentToActionAllocationReasoner

from dotenv import load_dotenv

In [35]:
openai.api_key = os.getenv("OPENAI_API_KEY")
llm_agent_to_action = LLMAgentToActionAllocationReasoner("prompts")

def apply_llm_action_costs(row):
    # Llamar a la función con los subgoals y condition de la fila actual
    agent, subgoals_disfavour, subgoals_favour, pddl_costs = llm_agent_to_action.llm_pddl_action_costs_for_agent_condition(
        str(row['subgoals']), row['condition']
    )
    
    # Devolver los valores en un diccionario para agregarlos como nuevas columnas
    return pd.Series({
        'agent': agent,
        'subgoals_disfavour': subgoals_disfavour,
        'subgoals_favour': subgoals_favour,
        'pddl_costs': pddl_costs
    })


In [41]:
# Aplicar la función a cada fila y asignar las nuevas columnas al DataFrame
df[['agent', 'subgoals_disfavour', 'subgoals_favour']] = df.apply(lambda row: apply_llm_action_costs(row)[:-1], axis=1)

The agent should disfavour these tasks:
None

The agent should favour the tasks:
['letter placed envelope', 'stamp put envelope', 'package picked_up loc2_entradaFME', 'records placed box', 'box brought warehouse']
["['letter placed envelope', 'stamp put envelope', 'package picked_up loc2_entradaFME', 'records placed box', 'box brought warehouse']"]
["['letter placed envelope'", "'stamp put envelope'", "'package picked_up loc2_entradaFME'", "'records placed box'", "'box brought warehouse']"]
{'attribute': "'stamp put envelope'_cost", 'agent': 'human', 'obj': "['letter placed envelope'", 'loc': "'package picked_up loc2_entradaFME'", 'value': 100}
The agent should disfavour these tasks:
cart picked_up loc3
package picked_up loc2_entradaFME
cart leaved loc3

The agent should favour the tasks:
letter placed envelope
stamp put envelope
records placed box
box brought warehouse
['cart picked_up loc3', 'package picked_up loc2_entradaFME', 'cart leaved loc3']
['cart picked_up loc3']
['package pi

In [45]:
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
df.drop(columns=['condition','subgoals']).to_csv(f"outputs/try_agent_reception_{timestr}.csv",index=False)